In [1]:
print("ok")

ok


1. Import Dependecies

In [7]:
# Import dependencies
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv

2. Pinecone cluster

In [3]:
load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [4]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

3. Doc Loader

In [9]:
# Extract Data from the PDF
# https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="./*.pdf",
                           loader_cls=PyPDFLoader
                           )
    documents = loader.load()

    return documents

